# Example of HTML capabilities of project inside IPython / Jupyter notebook

**Note**, this file used in tests, so it may contain additional test related logic.

## Contents

- [Initializing Environment](#Initializing-Environment)
- [Connect to odoo database](#Connect-to-odoo-database)
- [Session class](#Session-class)
- [Connect to odoo database (via session)](#Connect-to-odoo-database-via-session)
- [Module Utils plugin](#Module-Utils-plugin)
- [Get list of all registered objects / models](#Get-list-of-all-registered-objects-/-models)
- [Get object / model](#Get-object-/-model)
- [Getting information about available columns for Object / Model](#Getting-information-about-available-columns-for-Object)
- [Search for sale orders](#Search-for-sale-orders)
- [Display Sale orders as HTML table](#Display-Sale-orders-as-HTML-table)
- [Anyfield integration](#Anyfield-integration)
- [Nested HTMLTables](#Nested-HTMLTables)
- [Recordlist-elements-access](#Recordlist-elements-access)
- [Display one sale order as HTML Table](#Display-one-sale-order-as-HTML-Table)
- [Report service](#Report-service)

## Initializing Environment

In [1]:
# import extensions first (they modify Session and Client classes)
from openerp_proxy.ext.all import HField

# Enable module_utils plugin
import openerp_proxy.plugins.module_utils

# Import Client and Session classes
from openerp_proxy import (Client,
                           Session)

## Connect to odoo database

For connection to Odoo [Client](http://pythonhosted.org/openerp_proxy/module_ref/openerp_proxy.html#openerp_proxy.core.Client) class is used.
Below is example of it's usage.

In [2]:
cl = Client('localhost')  # connect to local instance of server
cl

login,None
Host,localhost
Protocol,xml-rpc
Port,8069
Database,None


Next we should check if our database is present on server.
For this purpose we should use Odoo's database service, which could be accessed via [Client.services.db](http://pythonhosted.org/openerp_proxy/module_ref/openerp_proxy.service.html#module-openerp_proxy.service.db).
And if database does not exists, we can create it.

In [3]:
# check if our demo database exists
if 'openerp_proxy_test_db' not in cl.services.db:
    # create demo database
    cl.services.db.create_db('admin', 'openerp_proxy_test_db', demo=True, lang='en_US')

And now we can [login](http://pythonhosted.org/openerp_proxy/module_ref/openerp_proxy.html#openerp_proxy.core.Client.login) to our database.

In [4]:
# login to created database
ldb = cl.login('openerp_proxy_test_db', 'admin', 'admin')  # all this arguments could be passed directly to Client constructor.

# and let's look how it is displayed in IPython
ldb

login,admin
Host,localhost
Protocol,xml-rpc
Port,8069
Database,openerp_proxy_test_db


*Note*, that ```ldb``` is new instance of [Client](http://pythonhosted.org/openerp_proxy/module_ref/openerp_proxy.html#openerp_proxy.core.Client) class, but with login credential. it can be used to interact with object service (models, documents, logic, ...)

## Session class

If You often need to connect to same databases, there are a [Session](http://pythonhosted.org/openerp_proxy/module_ref/openerp_proxy.html#openerp_proxy.session.Session) class,
which automaticaly save, most of your connections, made via [Session.connect](http://pythonhosted.org/openerp_proxy/module_ref/openerp_proxy.html#openerp_proxy.session.Session.connect) method in specified file.

In [5]:
# create session instance
session = Session('~/.openerp_proxy.local.json')  # default file path is '~/.openerp_proxy.json'

And there are option You may be interested in. It is 'store_passwords', which automaticaly saves password You have used for connection.

In [6]:
session.option('store_passwords', True)

True

We used ```openerp_proxy.Client``` class to create connection to database, so our session does not know anything about it. Let's add our connection to our session:

In [7]:
session.add_db(ldb)
session

DB URL,DB Index,DB Aliases
xml-rpc://admin@localhost:8069/openerp_proxy_test_db,1,ldb


And now we can get this connection from session by index, or by URL (look at the table above). But to simplify next connections, we may add aliase to this connection

In [8]:
session.aliase('ldb', ldb)
session

DB URL,DB Index,DB Aliases
xml-rpc://admin@localhost:8069/openerp_proxy_test_db,1,ldb


So, now, to get connection again we could just type ```session.ldb```.

Ok. initialization is done, and now we could save it.

In [9]:
session.save()

## Connect to odoo database via session

Let's now create new instance of session, and connect to created above database

In [10]:
session = Session('~/.openerp_proxy.local.json')  # default file path is '~/.openerp_proxy.json'
ldb = session.ldb
ldb

login,admin
Host,localhost
Protocol,xml-rpc
Port,8069
Database,openerp_proxy_test_db


## Module Utils plugin

Our database is clean, for next code, we need to install ```sale``` addon. For this, we have ```module_utils``` plugin, out-of-the box, which simplyfies work with modules. To enable this plugin, we just need to import it, and then we will have it in ```ldb.plugins``` property. This plugin was imported above.
This plugin extends ```ir.module.module``` model from client side, adding simple methods: ```install``` and ```upgrade``` to it for shorter syntax. If You're interested for code, look [here](https://github.com/katyukha/openerp-proxy/blob/master/openerp_proxy/plugins/module_utils.py), it is very simple!

So now, let's install ```sale``` module.  (Note that in most cases, **IPython autocompletition** work's fine)

In [11]:
ldb.plugins.module_utils.m_sale.install()

{'tag': 'reload', 'type': 'ir.actions.client'}

Congratulation! module was installed! Now we need only to refresh caches, to see new models in database.

In [12]:
ldb.clean_caches()

## Get list of all registered objects / models

To get list of registered models, just use *registered_objects* proerty of *Client* instance.
It returns list of all registered models in database. For example:

In [13]:
ldb.registered_objects

Name,System Name,Description
Account,account.account,False
Templates for Accounts,account.account.template,False
Account Type,account.account.type,False
account.addtmpl.wizard,account.addtmpl.wizard,"Add one more account from the template. With the 'nocreate' option, some accounts may not be created. Use this to add them later."
Account Aged Trial balance Report,account.aged.trial.balance,False
Analytic Account,account.analytic.account,False
Account Analytic Balance,account.analytic.balance,False
Account Analytic Chart,account.analytic.chart,False
Account Analytic Cost Ledger,account.analytic.cost.ledger,False
Account Analytic Cost Ledger For Journal Report,account.analytic.cost.ledger.journal.report,False


## Get object / model

*ldb* here represents database connection (*Client* class instance)
As told in help message above *.get_obj* method allows to get instance of specified
*Object* proxy, where Object means *model*, *document*.

In [14]:
so = ldb.get_obj('sale.order')
so

Model,sale.order
Client,xml-rpc://admin@localhost:8069/openerp_proxy_test_db
Name,Sales Order
Record count,8


Also it is posible to use shorter (dictionary style) syntax:

In [15]:
so = ldb['sale.order']
so

Model,sale.order
Client,xml-rpc://admin@localhost:8069/openerp_proxy_test_db
Name,Sales Order
Record count,8


And as result of using [*openerp_proxy.ext.sugar*](http://pythonhosted.org/openerp_proxy/module_ref/openerp_proxy.ext.html#module-openerp_proxy.ext.sugar) extension
(which is automaticaly imported in 'openerp_proxy.ext.all')
there are attribute-style access (which also support's IPython auto-completition):

In [16]:
so = ldb._sale_order
so

Model,sale.order
Client,xml-rpc://admin@localhost:8069/openerp_proxy_test_db
Name,Sales Order
Record count,8


## Getting information about available columns for Object

In [17]:
so.columns_info

Name,Disp. Name,Type,Required,Help
amount_tax,Taxes,float,None,The tax amount.
amount_total,Total,float,None,The total amount.
amount_untaxed,Untaxed Amount,float,None,The amount without tax.
client_order_ref,Customer Reference,char,None,None
company_id,Company,many2one,None,None
create_date,Creation Date,datetime,None,Date on which sales order is created.
currency_id,Currency,many2one,True,None
date_confirm,Confirmation Date,date,None,Date on which sales order is confirmed.
date_order,Date,date,True,None
fiscal_position,Fiscal Position,many2one,None,None


## Search for sale orders

In [18]:
# Standard search .search_records(domain)
so_list = so.search_records([])
so_list

Client,xml-rpc://admin@localhost:8069/openerp_proxy_test_db
Object,Object ('sale.order')
Record count,8


Also there are shorter syntax provided by [*openerp_proxy.ext.sugar*](http://pythonhosted.org/openerp_proxy/module_ref/openerp_proxy.ext.html#module-openerp_proxy.ext.sugar) extension:

In [19]:
so_list = so([])
so_list

Client,xml-rpc://admin@localhost:8069/openerp_proxy_test_db
Object,Object ('sale.order')
Record count,8


## Display Sale orders as HTML table

It is posible to represent RecordList as HTML table, with ability to highlight rows by specified conditions.
Also, when building result table, it is posible to display values of related fields, and even method calls. This functionality is implemented in [*openerp_proxy.ext.repr*](http://pythonhosted.org/openerp_proxy/module_ref/openerp_proxy.ext.html#module-openerp_proxy.ext.repr) module

In [20]:
# High light rows by condition
highlighters = {
    '#99FF99': lambda x: x.state == 'done',
    '#9999FF': lambda x: x.state == 'draft',
    '#FFFF99': lambda x: x.state == 'progress',
}

# Display as table.
# Note that prefetch method is used to fetch some set of fields with less RPC call.
# on big datasets it may speed up performance signifiantly.
# Each RecordList instance have related cache, which reduce need of reading data on each field get.
so_list.prefetch('id', 'name', 'partner_id', 'partner_id.email', 'state')
so_table = so_list.as_html_table(
    'id',
    'name',
    # _name attribute provides result of *name_search method:
    HField('partner_id._name', name='Partner name'),
    # silent=True means, if field cannot be found, not throw error
    HField('partner_id.email', name='Partner email', silent=True),
    # Also it is posible to display result of method calls
    # 'as_html_list()' is method of RecordList.
    ('order_line.as_html_list', 'Order lines'),
    'state',
    highlighters=highlighters,
)
so_table

id,name,Partner name,Partner email,Order lines,state
8,SO008,Millennium Industries,False,,draft
7,SO007,Luminous Technologies,False,,manual
6,SO006,Think Big Systems,info@thinkbig.com,,draft
5,SO005,Agrolait,info@agrolait.com,,draft
4,SO004,Millennium Industries,False,,draft
3,SO003,Chamber Works,info@chamberworks.com,,draft
2,SO002,Bank Wealthy and sons,email@wealthyandsons.com,,draft
1,SO001,Agrolait,info@agrolait.com,,sent


There also available to_csv method, which allow to represent table in csv format

In [21]:
so_table.to_csv()

/home/katyukha/projects/erp-proxy/examples/tmp/csv/tmp3_ozfz.csv

## Anyfield integration

This app has **experimental** integration with [Anyfield](https://pypi.python.org/pypi/anyfield) library.
For example it is posible to use Anyfield expressions in [RecordList.filter](http://pythonhosted.org/openerp_proxy/module_ref/openerp_proxy.orm.html#openerp_proxy.orm.record.RecordList.filter) method.

Ususaly You use lambdas to filter some records. Something like this:

In [22]:
partners = ldb._res_partner()  # find all partners in database
filtered_partners_l = partners.filter(lambda x: x.sale_order_ids.length >= 1)
filtered_partners_l

Client,xml-rpc://admin@localhost:8069/openerp_proxy_test_db
Object,Object ('res.partner')
Record count,6


But using [Anyfield](https://pypi.python.org/pypi/anyfield/) expressions may be simpler.
This library have [F](http://pythonhosted.org/anyfield/#anyfield.F) variable defined,
which could be used as starting point of [SField](http://pythonhosted.org/anyfield/#anyfield.SField) expressions.

In [23]:
from anyfield import F

# Imagine that F is record in partners recordlist. and pass expression based on this to filter method.
# It will automaticaly be converted to filter function
filtered_partners_f = partners.filter(F.sale_order_ids.length >= 1)
filtered_partners_f

Client,xml-rpc://admin@localhost:8069/openerp_proxy_test_db
Object,Object ('res.partner')
Record count,6


To check that resultes are same, let's just compare them:

In [24]:
assert sorted(filtered_partners_f.ids) == sorted(filtered_partners_l.ids), "Filtered partners must be same in both cases"

## Nested HTMLTables

Also it is posible to display nested html tables. For example, lets list partners with sale orders related to them

In [25]:
filtered_partners_l.as_html_table(
    'id',
    'name',
    'parent_id',
    HField('sale_order_ids.as_html_table',
           'Sale orders'),
)

id  name                     parent_id  Sale orders
----  ---------------------  -----------  -----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
   6  Agrolait                         0  <div class='panel panel-default'>
            
            
            <table class='table table-bordered table-condensed table-striped'>
                <tr style='border: none'>
                    
                         <th>id</th>
                    
                         <th>_name</th>
                    
                </tr>
                
                    

                    
                         <tr style='border:none'>
                    

                    
                         <td>5</td>
                    
                         <td>SO005</td>
                    

                    </tr>
                
                    

                    
                         <tr style='border:none'>
                    

                    
                         <td>1</td>
                    
                         <td>SO001</td>
                    

                    </tr>
                
            </table>
            <div class='panel-footer'>Total lines: 2</div>
        <div>
  11  Bank Wealthy and sons            0  <div class='panel panel-default'>
            
            
            <table class='table table-bordered table-condensed table-striped'>
                <tr style='border: none'>
                    
                         <th>id</th>
                    
                         <th>_name</th>
                    
                </tr>
                
                    

                    
                         <tr style='border:none'>
                    

                    
                         <td>2</td>
                    
                         <td>SO002</td>
                    

                    </tr>
                
            </table>
            <div class='panel-footer'>Total lines: 1</div>
        <div>
  18  Chamber Works                    0  <div class='panel panel-default'>
            
            
            <table class='table table-bordered table-condensed table-striped'>
                <tr style='border: none'>
                    
                         <th>id</th>
                    
                         <th>_name</th>
                    
                </tr>
                
                    

                    
                         <tr style='border:none'>
                    

                    
                         <td>3</td>
                    
                         <td>SO003</td>
                    

                    </tr>
                
            </table>
            <div class='panel-footer'>Total lines: 1</div>
        <div>
  15  Luminous Technologies            0  <div class='panel panel-default'>
            
            
            <table class='table table-bordered table-condensed table-strip

## Recordlist elements access

Recordlist supports access to containing elements via indexes

In [26]:
so_list[0]

Client,xml-rpc://admin@localhost:8069/openerp_proxy_test_db
Object,Object ('sale.order')
ID,8
Name,SO008


## Display one sale order as HTML Table

In [27]:
so_list[0].as_html('name',
                   'origin',
                   'partner_id',        # Will display Reacord instance representing partner related to this sale order
                   'partner_id._name',  # Will display result of 'name_get' called on partner
                   'partner_id.sale_order_ids.length')  # Display how many sale orders have this partner

partner_id.sale_order_ids.length,2
partner_id,"R(res.partner, 19)[Millennium Industries]"
partner_id._name,Millennium Industries
name,SO008
origin,False


In [28]:
so_list[0].as_html()  # Display all fields for firest sale order record

Invoice Address,"R(res.partner, 52)[Millennium Industries, Jacob Taylor]"
Confirmation Date,False
Create Invoice,manual
Customer,"R(res.partner, 19)[Millennium Industries]"
Contract / Analytic,False
Order Reference,SO008
Delivery Address,"R(res.partner, 52)[Millennium Industries, Jacob Taylor]"
Invoice on,order
Shop,"R(sale.shop, 1)[Your Company]"
Messages,RecordList(mail.message): length=2
Terms and conditions,False


## Report service

There is ``reports`` service available in Odoo, which allows to print reports.
Below example usage of it.

Access the service:

In [29]:
ldb.services.report

Get list of available reports:

In [30]:
ldb.services.report.available_reports

report service name,report name,report model,report help info
account.account.balance,Trial Balance,account.account,False
account.analytic.account.balance,Analytic Balance,account.analytic.account,False
account.analytic.account.cost_ledger,Cost Ledger,account.analytic.account,False
account.analytic.account.inverted.balance,Inverted Analytic Balance,account.analytic.account,False
account.analytic.account.journal,Analytic Journal,account.analytic.journal,False
account.analytic.account.quantity_cost_ledger,Cost Ledger (Only quantities),account.analytic.account,False
account.central.journal,Central Journal,account.journal.period,False
account.general.journal,General Journal,account.journal.period,False
account.general.ledger,General Ledger,account.account,False
account.general.ledger_landscape,General Ledger,account.account,False


Choose report name for current Odoo server version

In [31]:
from pkg_resources import parse_version as V
if ldb.server_version >= V("8.0"):
    report_name = 'sale.report_saleorder'
else:
    report_name = 'sale.order'

report_name

'sale.order'

Take a look on report object. It also has it's own HTML representation.

In [32]:
report = ldb.services.report[report_name]
report

Model,sale.order
Service name,sale.order
Name,Quotation / Order


And generate new report:

In [33]:
report_result = report.generate(so_list)
report_result